In [1]:
import pandas as pd
import re
import urllib.request
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import Transformer
from scheduler import TransformerScheduler

C:\Users\wintee\AppData\Local\Temp\ipykernel_24724\2596381722.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


[손실 함수 정의]
예제는 다중 클래스 분류 문제. 이때 레이블이 정수 형태이므로 손실 함수는 SparseCategoricalCrossentropy 사용

In [2]:
def loss_function(ans, pred):
    """
    다중 클래스 분류 문제를 위한 손실 함수 정의
    
    :param ans: 해당 데이터의 실제 정답
    :param pred: 모델이 생성해낸 예측 레이블
    :return: 손실값
    """
    ans = tf.reshape(ans, shape=(-1, MAX_LENGTH - 1))
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(ans, pred)
    mask = tf.cast(tf.not_equal(ans, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

[데이터 로드]
챗봇 데이터를 로드
학습 기반 토크나이저 사용을 위해 구두점 처리

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
print(f'샘플의 개수 : {len(train_data)}')

샘플의 개수 : 11823


In [5]:
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


In [6]:
# 구두점 제거 대신 띄어쓰기를 추가하여 다른 문자와 구분
# 정규식 사용하여 처리
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)
    
answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [7]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


[단어 집합 생성]
서브워드 텍스트 인코더를 사용하여 서브워드로 구성된 단어 집합 생성

In [8]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [9]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [10]:
print(f'START_TOKEN : {START_TOKEN}')
print(f'END_TOKEN : {END_TOKEN}')
print(f'VOCAB_SIZE : {VOCAB_SIZE}')

START_TOKEN : [8178]
END_TOKEN : [8179]
VOCAB_SIZE : 8180


[정수 인코딩과 패딩]
토크나이저의 .encode()를 사용하여 정수 인코딩

In [11]:
sample_string = questions[20]
tokenized_string = tokenizer.encode(sample_string)
print(f'원본 문장 : {sample_string}')
print(f'encode 후 : {tokenized_string}')
print(f'decode 후 : {tokenizer.decode(tokenized_string)}')

원본 문장 : 가스비 비싼데 감기 걸리겠어
encode 후 : [5766, 611, 3509, 141, 685, 3747, 849]
decode 후 : 가스비 비싼데 감기 걸리겠어


In [12]:
for token in tokenized_string:
    print(f'{token} ----> {tokenizer.decode([token])}')

5766 ----> 가스
611 ----> 비 
3509 ----> 비싼
141 ----> 데 
685 ----> 감기 
3747 ----> 걸리
849 ----> 겠어


In [13]:
MAX_LENGTH = 40

def encode_and_padding(inputs, outputs):
    """
    1. 토크나이저로 인코딩
    2. START_TOKEN, END_TOKEN 추가
    3. 패딩 수행
    
    :param inputs: 데이터 셋의 입력
    :param outputs: 데이터 셋의 출력
    :return: 인코딩된 입력과 출력 리스트
    """
    encoded_inputs, encoded_outputs = [], []
    
    for input_sentence, output_sentence in zip(inputs, outputs):
        encoded_inputs.append(START_TOKEN + tokenizer.encode(input_sentence) + END_TOKEN)
        encoded_outputs.append(START_TOKEN + tokenizer.encode(output_sentence) + END_TOKEN)
        
    encoded_inputs = tf.keras.preprocessing.sequence.pad_sequences(encoded_inputs, maxlen=MAX_LENGTH, padding='post')
    encoded_outputs = tf.keras.preprocessing.sequence.pad_sequences(encoded_outputs, maxlen=MAX_LENGTH, padding='post')
    
    return encoded_inputs, encoded_outputs

In [14]:
encoded_questions, encoded_answers = encode_and_padding(questions, answers)

print(f'질문 데이터의 크기 : {encoded_questions.shape}')
print(f'답변 데이터의 크기 : {encoded_answers.shape}')

print(f'0번 샘플 질문 데이터 : {encoded_questions[0]}')
print(f'0번 샘플 답변 데이터 : {encoded_answers[0]}')

질문 데이터의 크기 : (11823, 40)
답변 데이터의 크기 : (11823, 40)
0번 샘플 질문 데이터 : [8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
0번 샘플 답변 데이터 : [8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


[트랜스포머 만들기]
인풋 모양은 (2(인코더 입력, 디코더 입력), batch_size, MAX_LENGTH)을 의미

In [15]:
BATCH_SIZE = 64
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

transformer = Transformer(vocab_size=VOCAB_SIZE,
                          d_model=D_MODEL,
                          num_layers=NUM_LAYERS,
                          num_heads=NUM_HEADS,
                          d_ff=DFF,
                          dropout=DROPOUT)

transformer.build(input_shape=(2, BATCH_SIZE, MAX_LENGTH))

In [16]:
learning_rate = TransformerScheduler(d_model=D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

transformer.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [17]:
EPOCHS = 60
transformer.fit(x=(encoded_questions, encoded_answers[:, :-1]), y=encoded_answers[:, 1:], batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/60


185/185 [==============================] - 61s 245ms/step - loss: 1.5455 - accuracy: 0.0015
Epoch 2/60
185/185 [==============================] - 45s 242ms/step - loss: 1.3615 - accuracy: 0.0328
Epoch 3/60
185/185 [==============================] - 45s 241ms/step - loss: 1.1254 - accuracy: 0.0456
Epoch 4/60
185/185 [==============================] - 45s 242ms/step - loss: 1.0149 - accuracy: 0.0494
Epoch 5/60
185/185 [==============================] - 45s 243ms/step - loss: 0.9372 - accuracy: 0.0522
Epoch 6/60
185/185 [==============================] - 45s 246ms/step - loss: 0.8824 - accuracy: 0.0561
Epoch 7/60
185/185 [==============================] - 45s 241ms/step - loss: 0.8300 - accuracy: 0.0598
Epoch 8/60
185/185 [==============================] - 45s 242ms/step - loss: 0.7735 - accuracy: 0.0650
Epoch 9/60
185/185 [==============================] - 44s 240ms/step - loss: 0.7127 - accuracy: 0.0710
Epoch 10/60
185/185 [==============================] - 44s 240ms/step - loss: 0.6495

[평가]

In [18]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [19]:
def evaluate(sentence):
    print(sentence)
    sentence = preprocess_sentence(sentence)
    sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
    output = tf.expand_dims(START_TOKEN, 0)
    
    for i in range(MAX_LENGTH):
        predictions = transformer.predict((sentence, output), verbose=0)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
            
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

In [20]:
def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])
    
    print(f'Input: {sentence}')
    print(f'Output: {predicted_sentence}')
    
    return predicted_sentence

In [22]:
output = predict("공부 하기 싫어")

공부 하기 싫어
Input: 공부 하기 싫어
Output: 거리를 걸어보세요 .


In [23]:
output = predict("게임하고 싶어")

게임하고 싶어
Input: 게임하고 싶어
Output: 게임하세요 !


In [52]:
output = predict("다음 랩장은 누구일까요?")

다음 랩장은 누구일까요?
Input: 다음 랩장은 누구일까요?
Output: 계획 세우고 하세요 .
